Spectral Augmentation

In [ ]:

import os
import sys
from PIL import Image, ImageEnhance
Image.MAX_IMAGE_PIXELS = None

mypath = ""
myortomosaic = ""

def change_brightness(path, ortomosaic, brightness = 2):
    img = Image.open(os.path.join(path, ortomosaic))
    print(img.size, img.mode)
    enhancer = ImageEnhance.Brightness(img)
    print("illumination: "+ str(brightness))
        
    return enhancer.enhance(brightness).save(os.path.join(path,'Ilum_{}'.format(ortomosaic)))
        

def change_contrast(path, ortomosaic, contrast = 1.5):
    img = Image.open(os.path.join(path, ortomosaic))
    print(img.size, img.mode)
    enhancer = ImageEnhance.Contrast(img)
    print("contrast: "+ str(contrast))
    
    return enhancer.enhance(contrast).save(os.path.join(path,'Cont_{}'.format(ortomosaic)))
        
change_brightness(mypath,myortomosaic)
change_contrast(mypath,myortomosaic)



Geometric Augmentation

In [ ]:
mypath = ""
imagenes = 0

for f in os.listdir(mypath):
    fname, fext = os.path.splitext(f)
    #print(fext)
    if fext == '.png':
        i = Image.open(os.path.join(mypath, f))
        print(f, i.size, i.mode)
        #i.rotate(-90).convert('RGB').save(os.path.join(mypath,'n{}.tif'.format(fname)))
        #i.rotate(20).convert('RGB').save(os.path.join(mypath,'n{}.tif'.format(fname)))
        #i.transpose(Image.ROTATE_90).convert('RGB').save(os.path.join(mypath,'r{}.tif'.format(fname))) # rotacion de 90 grados (contrario a las manecillas del reloj)
        i.transpose(Image.FLIP_TOP_BOTTOM).convert('RGB').save(os.path.join(mypath,'v{}.png'.format(fname)))
        #i.transpose(Image.FLIP_LEFT_RIGHT).convert('RGB').save(os.path.join(mypath,'h{}.tif'.format(fname)))
    
        imagenes = imagenes + 1

print("processed images: "+ str((imagenes)))

Orthomosaics and Raster Masks Tessellation

In [ ]:

savedir = "imgs chips"
filename = "orthomosaic"

img = Image.open(filename)
width, height = img.size
start_pos = start_x, start_y = (0, 0)
cropped_image_size = w, h = (256,256)
#cropped_image_size = w, h = (1500,1500)
#cropped_image_size = w, h = (1024,1024)
#cropped_image_size = w, h = (2000,2000)
#cropped_image_size = w, h = (1500,1500)
#cropped_image_size = w, h = (1000,1000)
#cropped_image_size = w, h = (750,750)
#cropped_image_size = w, h = (600,600)
#cropped_image_size = w, h = (512,512)

frame_num = 1
for col_i in range(0, width, w):
    for row_i in range(0, height, h):
        crop = img.crop((col_i, row_i, col_i + w, row_i + h))
        save_to= os.path.join(savedir + "{:1}.png")
        crop.convert('RGB').save(save_to.format(frame_num))
        print(frame_num, end='', flush=True)
        frame_num += 1
print('number of img chips of ' + str(w) + 'x' + str(h) + 
      ' produced were:', frame_num-1)


savedir2 = "masks chips"
filename2 = "raster mask"

img = Image.open(filename2)
width, height = img.size
start_pos = start_x, start_y = (0, 0)

frame_num = 1
for col_i in range(0, width, w):
    for row_i in range(0, height, h):
        crop = img.crop((col_i, row_i, col_i + w, row_i + h))
        save_to= os.path.join(savedir2 + "{:1}.png")
        crop.save(save_to.format(frame_num))
        print(frame_num, end='', flush=True)
        frame_num += 1
print('number of msk chips of ' + str(w) + 'x' + str(h) + 
      ' produced were:', frame_num-1)

Imbalance check and removing of imbalance masks, parameter imb %

In [ ]:
import cv2
import numpy as np

path2 = "msk chips"
path = "img chips"

count = 0
imb = 5

print(len(os.listdir(path)))
tot_imgs = len(os.listdir(path))

for filename in (os.listdir(path)):
    #img = Image.open(path + '\\' + filename)
    img = cv2.imread(path + '/' + filename, cv2.IMREAD_GRAYSCALE)
    n_white_pix = np.sum(img == 255)
    n_black_pix = np.sum(img == 0)
    if (n_white_pix/n_black_pix)*100 < imb:
        count = count + 1
        #print('Number of white pixels:', n_white_pix)
        print(filename, '% of white:', round((n_white_pix/n_black_pix)*100,1))
        os.remove(path + '/' + filename)
        os.remove(path2 + '/' + filename)
print("-> deleted img-msk:", count)
print("-> remaining img-msk:", (tot_imgs-count))


Pairing chips into img,msk sole images

In [ ]:
myfolder1 = "imgs"
myfolder2 = 'msks'
myoutfolder = "dataset"

total_width = 512
max_height = 256
#total_width = 2048
#max_height = 1024
#total_width = 1024
#max_height = 512
#total_width = 1200 
#max_height = 600
#total_width = 1500 
#max_height = 750
#total_width = 3000 
#max_height = 1500

lst1 = []
lst2 = []
#formato = '.jpg'
formato = '.png'

new_im = Image.new('RGB', (total_width, max_height), color=(0,0,0))

for count, filename in enumerate(sorted(os.listdir(myfolder1))):
    (lst1.append(filename))

for count2, filename2 in enumerate(sorted(os.listdir(myfolder2))):
    (lst2.append(filename2))

for i in range(0,len(lst1)):
    im=Image.open(myfolder1+str(lst1[i])).convert("RGB")
    #im=Image.open(myfolder1+str(lst1[i]))
    img=Image.open(myfolder2+str(lst2[i])).convert("RGB")
    #img=Image.open(myfolder2+str(lst2[i]))
    new_im.paste(im, (0,0))
    new_im.paste(img, (max_height,0))
    new_im.save(myoutfolder + str(i+1) + formato)
print("number of paired img-msk:" + str(i+1) + ' imagenes de ' +
      str(max_height) +' pixeles en el formato:' + formato)


Random Spliting the dataset into train, validation and test datasets

In [ ]:
import os
source = "dataset"
dest1 = "validation"
dest2 = "test"

p1 = 0.2 # this is the percentage split value between train and validation-test

files = os.listdir(source)
import shutil
import numpy as np
for f in files:
    if np.random.rand(1) < p1:
        shutil.move(source + '/'+ f, dest1 + '/'+ f)

p2 = 0.5
files = os.listdir(dest1)
for f in files:
    if np.random.rand(1) < p2:
       shutil.move(dest1 + '/'+ f, dest2 + '/'+ f)
print("process finished obtaining",p1*100,"% for validation and",(p1/2)*100, "% for test")
